In [1]:
!pip install unsloth
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 4,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
df = pd.read_csv("/content/train.csv")
df.head()

,text,summary,cloze,cloze_gt,cloze_annotation,need_reasoning
0,"(Jan 30, 2020 7:00 PM) Fred Silverman, who st...",Fred Silverman Put a Series of Hits on All 3 M...,Fred Silverman Put a Series of Hits on All ___...,3,Trans( Three ),1
1,"(Mar 8, 2008 5:17 PM) Barack Obama easily won...",Obama Wins 7 Delegates in Wyo.,Obama Wins ____ Delegates in Wyo.,7,Copy( 7 ),0
2,"(Mar 12, 2014 6:32 PM CDT) They may be small,...",Scholar Finds 9 More Dead Sea Scrolls,Scholar Finds ____ More Dead Sea Scrolls,9,Trans( nine ),1
3,"(Apr 14, 2019 6:07 AM CDT) Babies wail as a n...","On Island Nation, Measles Has Killed 1.2K","On Island Nation, Measles Has Killed ____K",1.2,"Paraphrase( 1,200 , K )",1
4,"(Feb 26, 2015 8:11 AM) Italian surgeon Sergio...",Surgeon: We Could Transplant Human Head in 2017,Surgeon: We Could Transplant Human Head in ____,2017,Copy( 2017 ),0


In [6]:
prompts = pd.read_csv("/content/final_chain_of_thought.csv", encoding='cp1252')


In [ ]:
prompts.head()

,Category,Cloze,Article,Answer
0,Copy,Obama Wins _ Delegates in Wyo.,Barack Obama easily won the Wyoming caucuses t...,Obama won 7 delegates in Wyoming. Wyoming can ...
1,Transform,St. Louis Gun-Toters Get ____ Powerful Allies,Looks like the St. Louis couple who pointed gu...,The St. Louis couple gained two powerful defen...
2,Transform,Olympic Flame Doused ____ Times; Paris Relay C...,Paris police today extinguished the Olympic fl...,The Olympic flame is the subject of the paragr...
3,Paraphrase,"On Island Nation, Measles Has Killed __K",Babies wail as a nurse tries to reassure mothe...,Babies cry as a nurse comforts mothers vaccina...
4,Paraphrase,Scholar finds _ more dead sea scrolls,"They may be small, but they're still Dead Sea ...","Tiny but significant, nine new Dead Sea Scroll..."


In [7]:
prompt_basic = """"Task: Fill in the blank of this Sentence: "{}", with the appropriate numeral based on the following Article: "{}" Explanation: {}"""
prompt1 = prompt_basic.format(prompts.loc[0, "Cloze"], prompts.loc[0, "Article"], prompts.loc[0, "Answer"])# + EOS_TOKEN
prompt1


'"Task: Fill in the blank of this Sentence: "Obama Wins _ Delegates in Wyo.", with the appropriate numeral based on the following Article: "Barack Obama easily won the Wyoming caucuses today and added 7 delegates to his tally, Chris Cillizza reports in his Washington Post blog. Obama won 61% of the vote and Hillary Clinton won 38%, scoring 5 delegates. Obama campaign chief David Plouffe called the victory a very important win while Clinton\'s camp depicted the result as a suprisingly good near split in delegates. Both campaigns fired off memos over Iraq war plans as the Wyoming votes were being tallied. Clinton\'s blasted her rival for claiming to have a pullout plan even thoughdeparted adviser Samantha Power said the campaign had none. Obama\'s camp replied that he has made hard end dates about Iraq a centerpiece of his campaign." Explanation: Obama won 7 delegates in Wyoming. Wyoming can be abbriviated as Wyo. If someone wins their number is more important. In reference to a specific

In [8]:
#Scenario 1#
prompt_basic = """Task: Fill in the blank of this Sentence: "{}", with the appropriate numeral based on the following Article: "{}" Explanation: {}    """
prompt2 = prompt_basic.format(prompts.loc[0, "Cloze"], prompts.loc[0, "Article"], prompts.loc[0, "Answer"])
prompt2 += prompt_basic.format(prompts.loc[1, "Cloze"], prompts.loc[1, "Article"], prompts.loc[1, "Answer"])
prompt2 += prompt_basic.format(prompts.loc[3, "Cloze"], prompts.loc[3, "Article"], prompts.loc[3, "Answer"])
prompt2 += prompt_basic.format(prompts.loc[5, "Cloze"], prompts.loc[5, "Article"], prompts.loc[5, "Answer"])# + EOS_TOKEN
prompt2

'Task: Fill in the blank of this Sentence: "Obama Wins _ Delegates in Wyo.", with the appropriate numeral based on the following Article: "Barack Obama easily won the Wyoming caucuses today and added 7 delegates to his tally, Chris Cillizza reports in his Washington Post blog. Obama won 61% of the vote and Hillary Clinton won 38%, scoring 5 delegates. Obama campaign chief David Plouffe called the victory a very important win while Clinton\'s camp depicted the result as a suprisingly good near split in delegates. Both campaigns fired off memos over Iraq war plans as the Wyoming votes were being tallied. Clinton\'s blasted her rival for claiming to have a pullout plan even thoughdeparted adviser Samantha Power said the campaign had none. Obama\'s camp replied that he has made hard end dates about Iraq a centerpiece of his campaign." Explanation: Obama won 7 delegates in Wyoming. Wyoming can be abbriviated as Wyo. If someone wins their number is more important. In reference to a specific 

In [9]:
#Scenario 2#
prompt_basic = """Task: Fill in the blank of this Sentence: "{}", with the appropriate numeral based on the following Article: "{}" Explanation: {}    """
prompt3 = prompt_basic.format(prompts.loc[0, "Cloze"], prompts.loc[0, "Article"], prompts.loc[0, "Answer"])
prompt3 += prompt_basic.format(prompts.loc[1, "Cloze"], prompts.loc[1, "Article"], prompts.loc[1, "Answer"])
prompt3 += prompt_basic.format(prompts.loc[2, "Cloze"], prompts.loc[2, "Article"], prompts.loc[2, "Answer"])
prompt3 += prompt_basic.format(prompts.loc[3, "Cloze"], prompts.loc[3, "Article"], prompts.loc[3, "Answer"])
prompt3 += prompt_basic.format(prompts.loc[4, "Cloze"], prompts.loc[4, "Article"], prompts.loc[4, "Answer"])
prompt3 += prompt_basic.format(prompts.loc[5, "Cloze"], prompts.loc[5, "Article"], prompts.loc[5, "Answer"])#+ EOS_TOKEN
prompt3

'Task: Fill in the blank of this Sentence: "Obama Wins _ Delegates in Wyo.", with the appropriate numeral based on the following Article: "Barack Obama easily won the Wyoming caucuses today and added 7 delegates to his tally, Chris Cillizza reports in his Washington Post blog. Obama won 61% of the vote and Hillary Clinton won 38%, scoring 5 delegates. Obama campaign chief David Plouffe called the victory a very important win while Clinton\'s camp depicted the result as a suprisingly good near split in delegates. Both campaigns fired off memos over Iraq war plans as the Wyoming votes were being tallied. Clinton\'s blasted her rival for claiming to have a pullout plan even thoughdeparted adviser Samantha Power said the campaign had none. Obama\'s camp replied that he has made hard end dates about Iraq a centerpiece of his campaign." Explanation: Obama won 7 delegates in Wyoming. Wyoming can be abbriviated as Wyo. If someone wins their number is more important. In reference to a specific 

In [10]:
#Scenario 3#
prompt_basic = """Task: Fill in the blank of this Sentence: "{}", with the appropriate numeral based on the following Article: "{}" Explanation: {}    """
prompt4 = prompt_basic.format(prompts.loc[1, "Cloze"], prompts.loc[1, "Article"], prompts.loc[1, "Answer"])
prompt4 += prompt_basic.format(prompts.loc[2, "Cloze"], prompts.loc[2, "Article"], prompts.loc[2, "Answer"])
prompt4 += prompt_basic.format(prompts.loc[3, "Cloze"], prompts.loc[3, "Article"], prompts.loc[3, "Answer"])
prompt4 += prompt_basic.format(prompts.loc[4, "Cloze"], prompts.loc[4, "Article"], prompts.loc[4, "Answer"])#+ EOS_TOKEN
prompt4

'Task: Fill in the blank of this Sentence: "St. Louis Gun-Toters Get ____ Powerful Allies", with the appropriate numeral based on the following Article: "Looks like the St. Louis couple who pointed guns at protesters have two high-profile defenders: a governor and a president, the Washington Post reports. Missouri Gov. Mike Parson offered a whole-hearted defense Tuesday of Mark and Patricia McCloskey and said President Trump was also  getting involved  in their case. Declaring that the pair had  every right to protect their property,  Parson said Trump  doesn\'t like what he sees and the way these people are being treated  and added that Attorney General William Barr will likely  take a look  at the case with Trump." Explanation: The St. Louis couple gained two powerful defenders. The only numerical reference is two. The answer must be a digit. \'Two\' as a word is not a digit so this must be converted to the digit, \'2\'. The answer is 2     Task: Fill in the blank of this Sentence: "

In [11]:


prompt_basic = """Task: Fill in the blank of this Sentence: "{}", with the appropriate numeral based on the following Article: "{}" Explanation: {}    """

alpaca_prompt = """"You will be given a news article below. Following that, you will be given an input of a sentence with a blank.
Based on the news article, fill in the blank with an appropriate digit. Print the completed sentence"
First, observe the following examples with corresponding answers and explanations here:
### Examples:
{}

Now begin your task:
### Input:
{}

### Article:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["cloze"]
    articles   = examples["text"]
    outputs  = examples["summary"]
    texts = []
    for input, article, output in zip(inputs, articles, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(prompt3, input, article, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv', data_files="/content/train.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19422 [00:00<?, ? examples/s]

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/19422 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19,422 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 10,485,760


Step,Training Loss
1,2.295300
2,2.282600
3,2.297300
4,2.288100
5,2.256700
6,2.236700
7,2.182000
8,2.105000
9,2.075100
10,2.001700


In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format("",#np examples
        "Fred Silverman Put a Series of Hits on All ____ Major Networks", # input
        "(Jan 30, 2020  7:00 PM) Fred Silverman, who steered programming for each of the Big Three broadcast networks and brought All in the Family, Roots, Hawaii Five-O and other hit series and miniseries to television, died Thursday. He was 82. Silverman, who had been battling cancer, died at his home in the Pacific Palisades area of Los Angeles, the AP reports. Silverman's gift for picking winners prompted Time magazine to dub him  The Man with the Golden Gut  in a 1977 profile. As ABC's entertainment chief, Silverman turned the network’s fortunes around with shows including Roots, Rich Man, Poor Man and Charlie’s Angels. He had already brought success to CBS with an overhaul that included replacing country-themed series such as Green Acres with what advertisers considered more upscale and urban fare, including The Mary Tyler Moore Show, The Bob Newhart Show and Mannix. He couldn't repeat that success when he moved to NBC.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>"You will be given a news article below. Following that, you will be given an input of a sentence with a blank.\nBased on the news article, fill in the blank with an appropriate digit. Print the completed sentence"\nFirst, observe the following examples with corresponding answers and explanations here:\n### Examples:\n\n\nNow begin your task:\n### Input:\nFred Silverman Put a Series of Hits on All ____ Major Networks\n\n### Article:\n(Jan 30, 2020  7:00 PM) Fred Silverman, who steered programming for each of the Big Three broadcast networks and brought All in the Family, Roots, Hawaii Five-O and other hit series and miniseries to television, died Thursday. He was 82. Silverman, who had been battling cancer, died at his home in the Pacific Palisades area of Los Angeles, the AP reports. Silverman\'s gift for picking winners prompted Time magazine to dub him  The Man with the Golden Gut  in a 1977 profile. As ABC\'s entertainment chief, Silverman turned the network’s fo

In [16]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [17]:
#model.save_pretrained("lora_model") # Local saving
#tokenizer.save_pretrained("lora_model")
model.push_to_hub("anotherantonioflores/lora_model_prompt3", token = userdata.get('HF_TOKEN').strip()) # Online saving
tokenizer.push_to_hub("anotherantonioflores/lora_model_prompt3", token = userdata.get('HF_TOKEN').strip()) # Online saving

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Saved model to https://huggingface.co/anotherantonioflores/lora_model_prompt3


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]